# LangGraph Retrieval Agent

We can implement [Retrieval Agents](https://js.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents) in [LangGraph](https://js.langchain.com/docs/langgraph).

## Retriever

In [ ]:
import "chromadb";
import { CheerioWebBaseLoader } from "langchain/document_loaders/web/cheerio";
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { Chroma } from "@langchain/community/vectorstores/chroma";
import { OpenAIEmbeddings } from "@langchain/openai"; 

const urls = [
  "https://lilianweng.github.io/posts/2023-06-23-agent/",
  "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
  "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
];

const docs = await Promise.all(urls.map((url) => new CheerioWebBaseLoader(url).load()));
const docsList = docs.flat();

const textSplitter = new RecursiveCharacterTextSplitter({ chunkSize: 100, chunkOverlap: 50 });
const docSplits = await textSplitter.splitDocuments(docsList);

// Add to vectorDB
const vectorStore = await Chroma.fromDocuments(docSplits, new OpenAIEmbeddings(), {
  collectionName: "rag-chroma",
});
const retriever = vectorStore.asRetriever();

In [ ]:
import { createRetrieverTool } from "langchain/tools/retriever";
import { ToolExecutor } from "@langchain/langgraph/prebuilt";

const tool = createRetrieverTool(
  retriever,
  {
    name: "retrieve_blog_posts",
    description: "Search and return information about Lilian Weng blog posts."
  }
);
const tools = [tool];

const toolExecutor = new ToolExecutor({
  tools,
});